In [95]:
import dask.dataframe as dd
import pandas as pd
import pandas_ta as ta
import numpy as np
import os
import glob
import sys
from pathlib import Path
import pyarrow as pa


In [96]:
sector_file = 'stock_data/meta/ind_nifty500list_filtered_final.csv'
stock_sector = pd.read_csv(sector_file, dtype={'Symbol': str})
print(f"Loaded {len(stock_sector)} sector mappings")

# 2. Load Raw Stock Data
files = glob.glob('stock_data/raw/*.csv')
print(f"Found {len(files)} stock files")

dfs = []
for f in files:
    try:
        ticker = Path(f).stem
        if ticker not in stock_sector['Symbol'].values:
            print(f"Ticker {ticker} not found in sector mapping, skipping")
            continue

        # Read with date parsing and validation
        df = pd.read_csv(
            f,
            usecols=['Date', 'Close', 'Stock Splits'],
            parse_dates=['Date'],
            index_col='Date',
            dayfirst=True  # Important for non-US date formats
        )
        
        # Handle splits and calculate adjusted close
        df['Split Factor'] = (1 + df['Stock Splits']).replace(0, 1).cumprod()
        df['Adj Close'] = df['Close'] / df['Split Factor']
        
        dfs.append(df[['Adj Close']].rename(columns={'Adj Close': ticker}))
        
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")

# 3. Create Consolidated DataFrame
if not dfs:
    raise ValueError("No valid stock data processed")

master_df = pd.concat(dfs, axis=1)
print(f"Master DataFrame shape: {master_df.shape}")

# 4. Merge with Sector Information (preserve date index)
merged_df = (
    master_df.T
    .reset_index()
    .rename(columns={'index': 'Symbol'})
    .merge(stock_sector[['Symbol', 'Industry']],  # Verify actual column name
           on='Symbol',
           how='left')
    .set_index(['Industry', 'Symbol'])
    .T
    .sort_index(axis=1)
)

# Convert index to proper datetime
merged_df.index = pd.to_datetime(merged_df.index)
print("\nFinal Merged DataFrame:")
print(f"Index: {merged_df.index.name}")
print(f"Columns: {merged_df.columns.names}")
print(f"Shape: {merged_df.shape}")
print(merged_df.head(2))


Loaded 445 sector mappings
Found 501 stock files
Ticker CELLO not found in sector mapping, skipping
Ticker HONASA not found in sector mapping, skipping
Ticker TBOTEK not found in sector mapping, skipping
Ticker MANYAVAR not found in sector mapping, skipping
Ticker RRKABEL not found in sector mapping, skipping
Ticker IREDA not found in sector mapping, skipping
Ticker ADANIENSOL not found in sector mapping, skipping
Ticker SYRMA not found in sector mapping, skipping
Ticker SIGNATURE not found in sector mapping, skipping
Ticker INOXINDIA not found in sector mapping, skipping
Ticker RAINBOW not found in sector mapping, skipping
Ticker DELHIVERY not found in sector mapping, skipping
Ticker FIVESTAR not found in sector mapping, skipping
Ticker LLOYDSME not found in sector mapping, skipping
Ticker JnKBANK not found in sector mapping, skipping
Ticker INDGN not found in sector mapping, skipping
Ticker NSLNISP not found in sector mapping, skipping
Ticker TATATECH not found in sector mapping, ski

In [97]:
merged_df.head()

Industry                  Automobile and Auto Components              \
Symbol                                        APOLLOTYRE  ASAHIINDIA   
2022-01-10 00:00:00+05:30                     223.718369  547.070801   
2022-01-11 00:00:00+05:30                     223.092926  566.683289   
2022-01-12 00:00:00+05:30                     224.824921  559.700073   
2022-01-13 00:00:00+05:30                     226.075821  550.983398   
2022-01-14 00:00:00+05:30                     222.852371  570.843506   

Industry                                                                       \
Symbol                      BAJAJ-AUTO   BALKRISIND  BHARATFORG      BOSCHLTD   
2022-01-10 00:00:00+05:30  3186.578613  2343.807373  750.056519  16433.021484   
2022-01-11 00:00:00+05:30  3188.109131  2358.666016  752.206482  16307.836914   
2022-01-12 00:00:00+05:30  3191.216797  2387.699463  743.215576  16419.531250   
2022-01-13 00:00:00+05:30  3201.328369  2405.821777  750.545105  16411.558594   
2022-01-14 00:00:00+05:30  3184.444824  2425.745361  763.347412  16495.898438   

Industry                                                                      \
Symbol                         CEATLTD    CIEINDIA    CRAFTSMAN    EICHERMOT   
2022-01-10 00:00:00+05:30  1125.336304  221.650330  2311.613037  2736.037354   
2022-01-11 00:00:00+05:30  1126.414185  224.717316  2302.476807  2715.484375   
2022-01-12 00:00:00+05:30  1107.895264  219.849106  2310.520508  2736.426025   
2022-01-13 00:00:00+05:30  1116.664917  220.579315  2328.644775  2752.751953   
2022-01-14 00:00:00+05:30  1116.615723  221.455597  2324.125977  2740.118896   

Industry                   ... Telecommunication                           \
Symbol                     ...             ROUTE     TATACOMM    TEJASNET   
2022-01-10 00:00:00+05:30  ...       1767.905151  1421.090332  446.899994   
2022-01-11 00:00:00+05:30  ...       1817.629639  1415.016235  469.200012   
2022-01-12 00:00:00+05:30  ...       1830.158081  1490.009155  491.000000   
2022-01-13 00:00:00+05:30  ...       1862.771851  1478.339355  515.549988   
2022-01-14 00:00:00+05:30  ...       1843.613159  1478.100342  509.149994   

Industry                                Textiles                            \
Symbol                           TTML   ALOKINDS     KPRMILL       PAGEIND   
2022-01-10 00:00:00+05:30  276.350006  29.799999  717.887817  42242.050781   
2022-01-11 00:00:00+05:30  290.149994  30.000000  721.382324  42243.734375   
2022-01-12 00:00:00+05:30  275.649994  29.000000  733.932800  42989.968750   
2022-01-13 00:00:00+05:30  261.899994  28.750000  728.125122  42972.933594   
2022-01-14 00:00:00+05:30  248.850006  30.049999  744.514587  43183.343750   

Industry                                                      
Symbol                       TRIDENT         VTL  WELSPUNLIV  
2022-01-10 00:00:00+05:30  51.811363  515.310852  152.255203  
2022-01-11 00:00:00+05:30  54.382526  507.546356  148.519791  
2022-01-12 00:00:00+05:30  57.099224  510.023132  149.615524  
2022-01-13 00:00:00+05:30  59.912949  516.804810  153.251312  
2022-01-14 00:00:00+05:30  62.872211  528.766113  150.611633  

[5 rows x 441 columns]

In [98]:
def calculate_sma(data, window):
    return data.rolling(window=window).mean()

def calculate_ema(data, window):
    return data.ewm(span=window, adjust=False).mean()


In [99]:
def calculate_bollinger_bands(data, window, num_std_dev):
    sma = calculate_sma(data, window)
    rolling_std = data.rolling(window=window).std()
    upper_band = sma + (rolling_std * num_std_dev)
    lower_band = sma - (rolling_std * num_std_dev)
    return upper_band, lower_band

In [100]:
def calculate_rsi(data, window):
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [101]:
def calculate_macd(data, fast_window, slow_window, signal_window):
    ema_fast = calculate_ema(data, fast_window)
    ema_slow = calculate_ema(data, slow_window)
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    return macd_line, signal_line, macd_histogram

In [102]:
def approximate_atr(data, window):
    return data.diff().abs().rolling(window=window).mean()

In [103]:
merged_df.head()

Industry                  Automobile and Auto Components              \
Symbol                                        APOLLOTYRE  ASAHIINDIA   
2022-01-10 00:00:00+05:30                     223.718369  547.070801   
2022-01-11 00:00:00+05:30                     223.092926  566.683289   
2022-01-12 00:00:00+05:30                     224.824921  559.700073   
2022-01-13 00:00:00+05:30                     226.075821  550.983398   
2022-01-14 00:00:00+05:30                     222.852371  570.843506   

Industry                                                                       \
Symbol                      BAJAJ-AUTO   BALKRISIND  BHARATFORG      BOSCHLTD   
2022-01-10 00:00:00+05:30  3186.578613  2343.807373  750.056519  16433.021484   
2022-01-11 00:00:00+05:30  3188.109131  2358.666016  752.206482  16307.836914   
2022-01-12 00:00:00+05:30  3191.216797  2387.699463  743.215576  16419.531250   
2022-01-13 00:00:00+05:30  3201.328369  2405.821777  750.545105  16411.558594   
2022-01-14 00:00:00+05:30  3184.444824  2425.745361  763.347412  16495.898438   

Industry                                                                      \
Symbol                         CEATLTD    CIEINDIA    CRAFTSMAN    EICHERMOT   
2022-01-10 00:00:00+05:30  1125.336304  221.650330  2311.613037  2736.037354   
2022-01-11 00:00:00+05:30  1126.414185  224.717316  2302.476807  2715.484375   
2022-01-12 00:00:00+05:30  1107.895264  219.849106  2310.520508  2736.426025   
2022-01-13 00:00:00+05:30  1116.664917  220.579315  2328.644775  2752.751953   
2022-01-14 00:00:00+05:30  1116.615723  221.455597  2324.125977  2740.118896   

Industry                   ... Telecommunication                           \
Symbol                     ...             ROUTE     TATACOMM    TEJASNET   
2022-01-10 00:00:00+05:30  ...       1767.905151  1421.090332  446.899994   
2022-01-11 00:00:00+05:30  ...       1817.629639  1415.016235  469.200012   
2022-01-12 00:00:00+05:30  ...       1830.158081  1490.009155  491.000000   
2022-01-13 00:00:00+05:30  ...       1862.771851  1478.339355  515.549988   
2022-01-14 00:00:00+05:30  ...       1843.613159  1478.100342  509.149994   

Industry                                Textiles                            \
Symbol                           TTML   ALOKINDS     KPRMILL       PAGEIND   
2022-01-10 00:00:00+05:30  276.350006  29.799999  717.887817  42242.050781   
2022-01-11 00:00:00+05:30  290.149994  30.000000  721.382324  42243.734375   
2022-01-12 00:00:00+05:30  275.649994  29.000000  733.932800  42989.968750   
2022-01-13 00:00:00+05:30  261.899994  28.750000  728.125122  42972.933594   
2022-01-14 00:00:00+05:30  248.850006  30.049999  744.514587  43183.343750   

Industry                                                      
Symbol                       TRIDENT         VTL  WELSPUNLIV  
2022-01-10 00:00:00+05:30  51.811363  515.310852  152.255203  
2022-01-11 00:00:00+05:30  54.382526  507.546356  148.519791  
2022-01-12 00:00:00+05:30  57.099224  510.023132  149.615524  
2022-01-13 00:00:00+05:30  59.912949  516.804810  153.251312  
2022-01-14 00:00:00+05:30  62.872211  528.766113  150.611633  

[5 rows x 441 columns]

In [104]:
def add_enhanced_features(df):
    """Add only the most significant features for return_ratio prediction"""
    print(f"DataFrame structure: {df.columns.nlevels} levels")
    print(f"Column levels: {[name for name in df.columns.names]}")
    
    # Store new columns as tuples first, then create a new DataFrame at the end
    new_columns = {}
    
    # Process each industry and symbol
    for industry in df.columns.levels[0]:  # Iterate over industries
        symbols = df[industry].columns  # Get symbols for this industry
        
        for symbol in symbols:  # Iterate over symbols within each industry
            try:
                print(f"Adding significant features for {industry} - {symbol}")
                
                # Get base price data
                price = df[(industry, symbol)]
                
                #--------------------------------------------------------------
                # 1. ESSENTIAL PRICE-BASED FEATURES
                #--------------------------------------------------------------
                
                # Original price data
                new_columns[(industry, symbol, 'price')] = price
                
                # Key return timeframes (most significant for prediction)
                new_columns[(industry, symbol, 'return_1d')] = price.pct_change(1)
                new_columns[(industry, symbol, 'return_5d')] = price.pct_change(5)
                new_columns[(industry, symbol, 'return_21d')] = price.pct_change(21)  # ~1 month
                
                # Log return (better statistical properties)
                new_columns[(industry, symbol, 'log_return')] = np.log(price / price.shift(1))
                
                #--------------------------------------------------------------
                # 2. KEY TECHNICAL INDICATORS
                #--------------------------------------------------------------
                
                # Most important moving averages
                new_columns[(industry, symbol, 'SMA_20')] = price.rolling(window=20).mean()
                new_columns[(industry, symbol, 'SMA_50')] = price.rolling(window=50).mean()
                
                # Exponential Moving Averages (EMA)
                new_columns[(industry, symbol, 'EMA_10')] = price.ewm(span=10, adjust=False).mean()
                new_columns[(industry, symbol, 'EMA_20')] = price.ewm(span=20, adjust=False).mean()
                new_columns[(industry, symbol, 'EMA_50')] = price.ewm(span=50, adjust=False).mean()
                
                # Weighted Moving Averages (WMA)
                # For WMA, more recent prices receive higher weights
                for window in [10, 20, 50]:
                    weights = np.arange(1, window + 1)
                    wma = price.rolling(window=window).apply(
                        lambda x: np.sum(weights * x) / weights.sum(), raw=True
                    )
                    new_columns[(industry, symbol, f'WMA_{window}')] = wma
                
                # Relative position to moving averages (stronger than raw MA values)
                new_columns[(industry, symbol, 'dist_from_sma20')] = (price - new_columns[(industry, symbol, 'SMA_20')]) / new_columns[(industry, symbol, 'SMA_20')]
                new_columns[(industry, symbol, 'dist_from_sma50')] = (price - new_columns[(industry, symbol, 'SMA_50')]) / new_columns[(industry, symbol, 'SMA_50')]
                new_columns[(industry, symbol, 'dist_from_ema20')] = (price - new_columns[(industry, symbol, 'EMA_20')]) / new_columns[(industry, symbol, 'EMA_20')]
                
                # MACD (strong momentum indicator)
                ema12 = price.ewm(span=12, adjust=False).mean()
                ema26 = price.ewm(span=26, adjust=False).mean()
                macd_line = ema12 - ema26
                signal_line = macd_line.ewm(span=9, adjust=False).mean()
                new_columns[(industry, symbol, 'MACD')] = macd_line
                new_columns[(industry, symbol, 'MACD_signal')] = signal_line
                
                # RSI (key overbought/oversold indicator)
                delta = price.diff()
                gain = delta.where(delta > 0, 0)
                loss = -delta.where(delta < 0, 0)
                avg_gain = gain.rolling(window=14).mean()
                avg_loss = loss.rolling(window=14).mean()
                rs = avg_gain / avg_loss.replace(0, np.finfo(float).eps)  # Avoid division by zero
                new_columns[(industry, symbol, 'RSI')] = 100 - (100 / (1 + rs))
                
                # Volatility (key risk measure)
                new_columns[(industry, symbol, 'volatility_20d')] = price.pct_change().rolling(20).std()
                
                # Bollinger Band Width (volatility measure)
                sma20 = new_columns[(industry, symbol, 'SMA_20')]
                std20 = price.rolling(window=20).std()
                upper_band = sma20 + (std20 * 2)
                lower_band = sma20 - (std20 * 2)
                new_columns[(industry, symbol, 'BB_width')] = (upper_band - lower_band) / sma20
                
                # Moving average crossovers (strong trend signals)
                new_columns[(industry, symbol, 'ema_cross_signal')] = (
                    (new_columns[(industry, symbol, 'EMA_10')] > new_columns[(industry, symbol, 'EMA_20')]).astype(int)
                )
                
                #--------------------------------------------------------------
                # 3. CALENDAR EFFECTS
                #--------------------------------------------------------------
                
                # Month-end effect (significant calendar anomaly)
                new_columns[(industry, symbol, 'is_month_end')] = pd.Series(df.index).dt.is_month_end.astype(int).values
                
                #--------------------------------------------------------------
                # 4. TARGET VARIABLE
                #--------------------------------------------------------------
                
                new_columns[(industry, symbol, 'return_ratio')] = price.pct_change()
                
            except Exception as e:
                print(f"Failed to process {industry} - {symbol}: {e}")
                continue
    
    # Create a new DataFrame with the proper MultiIndex structure
    result_df = pd.DataFrame(new_columns, index=df.index)
    
    # Ensure proper MultiIndex structure
    result_df.columns = pd.MultiIndex.from_tuples(result_df.columns)
    
    #--------------------------------------------------------------
    # 5. INDUSTRY-RELATIVE FEATURES (highly significant)
    #--------------------------------------------------------------
    
    # Calculate industry averages for each day
    industry_daily_returns = {}
    for industry in result_df.columns.levels[0]:
        industry_returns = pd.DataFrame()
        
        for symbol in result_df[industry].columns:
            if (industry, symbol, 'return_ratio') in result_df.columns:
                industry_returns[symbol] = result_df[(industry, symbol, 'return_ratio')]
        
        if not industry_returns.empty:
            industry_daily_returns[industry] = industry_returns.mean(axis=1)
    
    # Calculate relative performance metrics (most predictive cross-sectional features)
    for industry in industry_daily_returns:
        industry_avg = industry_daily_returns[industry]
        
        for symbol in result_df[industry].columns:
            try:
                if (industry, symbol, 'return_ratio') in result_df.columns:
                    symbol_return = result_df[(industry, symbol, 'return_ratio')]
                    
                    # Relative performance vs industry (key factor)
                    result_df[(industry, symbol, 'rel_to_industry')] = symbol_return - industry_avg
                    
                    # Return Z-score (statistical significance of returns)
                    result_df[(industry, symbol, 'return_z_score')] = (
                        (symbol_return - industry_avg) / industry_returns.std(axis=1).replace(0, np.finfo(float).eps)
                    )
                    
                    # Cumulative 10-day industry-relative performance (momentum)
                    result_df[(industry, symbol, 'cum_rel_perf_10d')] = (
                        (1 + (symbol_return - industry_avg)).rolling(10).apply(lambda x: np.prod(x) - 1, raw=True)
                    )
            except Exception as e:
                print(f"Failed to calculate relative features for {industry} - {symbol}: {e}")
    
    return result_df
# Apply the function to your DataFrame
enhanced_df = add_enhanced_features(merged_df)

# Save the enhanced DataFrame
enhanced_df.ffill().bfill().dropna().to_parquet(
    'stock_data/processed/merged_stock_data_with_enhanced_features.parquet',
    engine='pyarrow',
    index=True,
    compression='snappy'
)

DataFrame structure: 2 levels
Column levels: ['Industry', 'Symbol']
Adding significant features for Automobile and Auto Components - APOLLOTYRE
Adding significant features for Automobile and Auto Components - ASAHIINDIA
Adding significant features for Automobile and Auto Components - BAJAJ-AUTO
Adding significant features for Automobile and Auto Components - BALKRISIND
Adding significant features for Automobile and Auto Components - BHARATFORG
Adding significant features for Automobile and Auto Components - BOSCHLTD
Adding significant features for Automobile and Auto Components - CEATLTD
Adding significant features for Automobile and Auto Components - CIEINDIA
Adding significant features for Automobile and Auto Components - CRAFTSMAN
Adding significant features for Automobile and Auto Components - EICHERMOT
Adding significant features for Automobile and Auto Components - ENDURANCE
Adding significant features for Automobile and Auto Components - EXIDEIND
Adding significant features for 

In [105]:
enhanced_df[100:105]

Automobile and Auto Components                      \
                                              APOLLOTYRE                       
                                                   price return_1d return_5d   
2022-06-07 00:00:00+05:30                     202.934189 -0.023159 -0.034120   
2022-06-08 00:00:00+05:30                     203.992661  0.005216 -0.035706   
2022-06-09 00:00:00+05:30                     200.672974 -0.016274 -0.060162   
2022-06-10 00:00:00+05:30                     192.830780 -0.039079 -0.081997   
2022-06-13 00:00:00+05:30                     182.438721 -0.053892 -0.121816   

                                                                         \
                                                                          
                          return_21d log_return      SMA_20      SMA_50   
2022-06-07 00:00:00+05:30   0.091898  -0.023431  205.221900  195.929170   
2022-06-08 00:00:00+05:30   0.066667   0.005202  206.008523  196.399700   
2022-06-09 00:00:00+05:30   0.065934  -0.016407  206.612323  196.776895   
2022-06-10 00:00:00+05:30   0.022449  -0.039864  206.600295  196.978963   
2022-06-13 00:00:00+05:30  -0.055071  -0.055399  205.601981  197.001094   

                                                               ...  \
                                                               ...   
                               EMA_10      EMA_20      EMA_50  ...   
2022-06-07 00:00:00+05:30  207.655217  205.059527  199.674058  ...   
2022-06-08 00:00:00+05:30  206.989297  204.957921  199.843415  ...   
2022-06-09 00:00:00+05:30  205.840875  204.549830  199.875946  ...   
2022-06-10 00:00:00+05:30  203.475403  203.433730  199.599665  ...   
2022-06-13 00:00:00+05:30  199.650552  201.434206  198.926687  ...   

                                 Textiles                            \
                               WELSPUNLIV                             
                          dist_from_sma50 dist_from_ema20      MACD   
2022-06-07 00:00:00+05:30       -0.134138       -0.011330 -3.126888   
2022-06-08 00:00:00+05:30       -0.101663        0.017729 -2.671786   
2022-06-09 00:00:00+05:30       -0.077121        0.036237 -2.157638   
2022-06-10 00:00:00+05:30       -0.054479        0.051497 -1.611035   
2022-06-13 00:00:00+05:30       -0.119497       -0.023352 -1.605437   

                                                                           \
                                                                            
                          MACD_signal        RSI volatility_20d  BB_width   
2022-06-07 00:00:00+05:30   -4.581981  52.819812       0.060777  0.171931   
2022-06-08 00:00:00+05:30   -4.199942  61.142868       0.045471  0.170195   
2022-06-09 00:00:00+05:30   -3.791481  51.079136       0.045085  0.181298   
2022-06-10 00:00:00+05:30   -3.355392  55.871888       0.045082  0.199199   
2022-06-13 00:00:00+05:30   -3.005401  53.859362       0.047568  0.189776   

                                                                      
                                                                      
                          ema_cross_signal is_month_end return_ratio  
2022-06-07 00:00:00+05:30                0            0    -0.029696  
2022-06-08 00:00:00+05:30                0            0     0.031317  
2022-06-09 00:00:00+05:30                0            0     0.022084  
2022-06-10 00:00:00+05:30                0            0     0.020257  
2022-06-13 00:00:00+05:30                0            0    -0.073461  

[5 rows x 10584 columns]